# Traitement d'un jeu d'articles de recherche

In [ ]:
import os

# Chargement des données

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
filepath = "../pdf" #Spécification du chemin d'accès. peut être amené à changer selon l'espace de travail
print(os.listdir(filepath)) #On affiche la liste de tous les fichiers présents dans le dossier, pour vérifier que notre programme les trouve

Le module PDFLoader nous retourne une liste de documents, chacun contenant une chaîne de caractères par page ainsi que les métadonnées du document ddans un dictionnaire. Il faut donc rassembler toutes les pages de l'article dans la même chaîne de caractères

In [ ]:
texts = {}
docs = {}

#On utilise des dictionnaires pour nos variables afin de pouvoir récupérer toutes les données d'un document à l'aide de son nom

for i in os.listdir(filepath): #Execute les instructions pour chaque fichier 
    loader = PyPDFLoader(filepath+'/'+i)
    docs[i] = loader.load() 
    texts[i] = ""
    for doc in docs[i]:
        texts[i] += doc.page_content #Concaténation
    texts[i] = texts[i].replace("\n", " ") #Mise en forme en un paragraphe
    print(i," - Number of pages:", len(docs[i]),"  Number of characters:", len(texts[i]))

Les caractères représentant la nouvelle ligne et le nouveau paragraphe sont tous les deux "\n". Il est sans doutes plus judicieux de traiter le pdf comme un énorme paragraphe pour le traitement qui va suivre

# Découpage des textes en plusieurs chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
chunk_size = 5000  # Maximum size of chunks
chunk_overlap = 200  # Overlap in characters between chunks

separators = [".", " ", ""]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=separators,
    keep_separator=False, 
) #Instanciation du "découpeur" des textes

In [ ]:
splits = {} #Création d'une liste de chunks par texte
for i in texts.keys():
    splits[i] = text_splitter.split_text(texts[i])
    print(i," - Number of text chunks:", len(splits[i]))

# Création de la collection Chroma

In [ ]:
import chromadb
from chromadb.config import Settings

In [ ]:
client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",
                                    persist_directory="db/"
                                )) 

Si vous voulez récupérer une collection déjà existente, éxecutez la cellule ci-dessous

In [ ]:
collection = client.get_or_create_collection(name="Articles") #Création de la base de données vectorielle

Si vous voulez recréer une base de données vectorielle, éxecutez la cellule ci-dessous

In [ ]:
client.delete_collection(name="Articles")
collection = client.create_collection(name="Articles")

Fonction permettant d'associer à quelle(s) page(s) se trouve les chunks créés plus tôt

In [ ]:
def sources(nom,splits,doc,text):
    output = []
    S = len(doc[0].page_content)-1
    pages = []
    pages.append(S)
    for page in doc[1::]:
        S += len(page.page_content)
        pages.append(S)
    for i in splits:
        ind = text.find(i)
        d = 0
        f = 0
        while pages[d] < ind : d+=1
        while pages[f] < (ind+len(i)-1) : f+=1
        output.append({"source": f"{nom} pages {d+1}-{f+1}"})
    return output

Ajout des articles à la collection s'ils n'y sont pas déjà (cas d'une collection récupérée)
La source et la page de chaque chunk sont stockés dans les métadonnées de chaque vecteur, à la clé "source"

In [ ]:
for i in splits.keys():
    try :
        collection.add(
            documents=splits[i],
            metadatas = sources(i,splits[i],docs[i],texts[i]),
            ids = [f"{i}_{j}" for j in range(len(splits[i]))]
        )
        print(f"{i} ajouté avec succès")
    except :
        print("Cet article est déjà dans la collection. Si vous avez changé les paramètres de tokenisation, merci de supprimer la collection et d'en créer une nouvelle.")

Test pour vérifier la présence de chunks dans la base de données vectorielle

In [ ]:
id = 52
print("Index:\n", collection.get()["ids"][id])
print("Text:\n", collection.get()["documents"][id])
print("Embedding vector:\n", collection.get(include=["embeddings"])["embeddings"][id])
print("Metadata:\n", collection.get()["metadatas"][id])

# Récupération des articles

Une requête est faite à la base de données vectorielle : en lui donnant une question, ou une phrase sur ce que l'on veut obtenir, la requête va nous retourner les chunks les plus pertinent (autrement dit les vecteurs les plus proches du vecteur de notre requête)

In [ ]:
results = collection.query(
    query_texts=["What is the classical sample preparation method to see microstructure of steel ?"], #La question que l'on souhaite poser
    n_results=10 #Le nombre de résultats pertinents que l'on souhaite
)

Sources = list(set([i["source"] for i in results["metadatas"][0]]))
Distances = [results["distances"][0][[i["source"] for i in results["metadatas"][0]].index(i)] for i in Sources]
Tri = [(x,y) for y,x in sorted(zip(Distances,Sources))] #Classement des résultats selon la distance : plus elle est petite, plus l'élément est pertinent

for i in Tri:
    print(f" Source : {i[0]}  Distance : {i[1]}")

Conversion de la base de données vectorielle pour une utilisation avec un LLM

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
vdb = Chroma(client=client,collection_name="Articles",persist_directory="db/") #Récupération de la collection, mise au bon format
vdb.persist()
retriever = vdb.as_retriever(search_type="similarity", search_kwargs={"k": 10})

Test de requête pour vérifier le fonctionnement

In [ ]:
query = "What is the classical sample preparation method to see microstructure of steel"
contexts = retriever.invoke(query)
list(set([contexts[i].metadata["source"] for i in range(len(contexts))]))


# Génération de réponse avec ChatGPT

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI

Instantiation du chat avec GPT4

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

Création d'un modèle qui va utiliser les variables "context" et "question" pour faire une requête à ChatGPT

In [ ]:
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use five sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

rag_prompt = PromptTemplate.from_template(template)

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

Création de notre RAG : L'argument passé en entrée sera la question, et il utilisera le retriever pour faire la requête à la base de données vectorielle. Il utilisera le prompt créé plus tôt pour faire la requête à chatGPT via l'objet llm instancié plus tôt

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
query = "What is the classical sample preparation method to see microstructure of steel ?"
answer = rag_chain.invoke(query)

In [ ]:
print(answer)